In [6]:
import json
from deepdiff import DeepDiff
from pprint import pprint

stg_data_path = "../temp_file/stg_product.jsons"
prod_data_path = "../temp_file/prod_product.jsons"

def convert_json(file_path):
    with open (file_path, "r") as myfile:
        data=myfile.read().replace("}\n{", "},\n{")
        data = "[" + data + "]"
        data = json.loads(data)
        return data

# write missing products to csv

# all_rids = []  
# with open(output_filename, "w") as output:
#     writer = csv.writer(output, lineterminator='\n')
#     for val in all_rids:
#         writer.writerow([val])
    
def compare_products(stg_data, prod_data):
    print("Comparing Data")
    for stg_product in stg_data:
        stg_product_id = stg_product['externalID']        
#         get stg product info
        for prod_product in prod_data:
            prod_product_id = prod_product['externalID']
            if stg_product_id == prod_product_id:
                ddiff = DeepDiff(stg_product, prod_product, ignore_order=True)
                print(ddiff)

    
    

def main():
    new_stg_data = convert_json(stg_data_path)
    new_prod_data = convert_json(prod_data_path)
    
    compare_products(new_stg_data, new_prod_data)
    
main()
    

Comparing Data


NameError: name 't1' is not defined